In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import ipywidgets as widgets

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
#city_data_df.head()

In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
gmaps.configure(api_key=g_key)

In [8]:
#Get the latitidue and longitude of each city
locations = city_data_df[["Lat","Lng"]]
#set any negative temperatures to 0F (shouldn't it be better to use Kelvin which is an absolute temp?)
city_data_df["Max Temp"] = city_data_df["Max Temp"].clip(lower=0)
#Get the max temperatures of each city
max_temp = city_data_df["Max Temp"]
# assign the gmaps figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# assign the heatmap
heatmap_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating = False, 
                                    max_intensity=300,point_radius=4)
# Add the heatmap layer
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
winds = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=winds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [31]:
temp_range_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                (city_data_df["Max Temp"] >= min_temp)]
temp_range_df = temp_range_df.dropna()
temp_range_df.head(5)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
13,13,Castro,BR,2021-04-05 18:53:04,-24.7911,-50.0119,73.2,51,35,6.96
15,15,Mahebourg,MU,2021-04-05 18:53:05,-20.4081,57.7000,78.8,83,40,10.36
16,16,Pisco,PE,2021-04-05 18:53:05,-13.7000,-76.2167,77.0,65,0,13.80
26,26,Marawi,PH,2021-04-05 18:53:08,7.9986,124.2928,75.0,89,100,5.01
29,29,Minab,IR,2021-04-05 18:53:09,27.1467,57.0801,75.2,69,0,6.91


In [32]:
temp_range_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119 entries, 13 to 566
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     119 non-null    int64  
 1   City        119 non-null    object 
 2   Country     119 non-null    object 
 3   Date        119 non-null    object 
 4   Lat         119 non-null    float64
 5   Lng         119 non-null    float64
 6   Max Temp    119 non-null    float64
 7   Humidity    119 non-null    int64  
 8   Cloudiness  119 non-null    int64  
 9   Wind Speed  119 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 10.2+ KB


In [34]:
hotel_df = temp_range_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
13,Castro,BR,73.20,-24.7911,-50.0119,
15,Mahebourg,MU,78.80,-20.4081,57.7000,
16,Pisco,PE,77.00,-13.7000,-76.2167,
26,Marawi,PH,75.00,7.9986,124.2928,
29,Minab,IR,75.20,27.1467,57.0801,
33,Mar Del Plata,AR,79.00,-38.0023,-57.5575,
47,Saint-Joseph,RE,78.80,-21.3667,55.6167,
48,Catalao,BR,77.22,-18.1658,-47.9464,
49,Antofagasta,CL,75.00,-23.6500,-70.4000,
51,Inirida,CO,78.44,3.8653,-67.9239,
